In [1]:
import jupyterthemes as jt
from jupyterthemes import jtplot
jtplot.style(theme='onedork')
!jt -t onedork -ofs 12 -dfs 12 -fs 12 -cellw 100% -T # onedork, grade3
#!jt -r
# source activate rdkit && 

In [2]:
import re, os, glob
import image_slicer
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="5,6,7,8"
os.environ["CUDA_VISIBLE_DEVICES"]="8"

In [3]:
from fastai import *
from fastai.vision import *
from fastai.core import *
from fastai.callbacks.hooks import *
from fastai.metrics import error_rate
import matplotlib.pyplot as plt
import re, os, glob
import torch
from PIL import Image

In [4]:
from patches import extract_patches_2d
from patches import reconstruct_from_patches_2d

In [5]:
base_path = os.path.join('/mnt', 'sdf')
path_to_images = os.path.join(base_path, 'training_data')
path_to_dfs = os.path.join(base_path, 'training_data', 'dfs')

In [6]:
# create a single dataframe with all file names and labels
dfs_filenames = glob.glob(os.path.join(path_to_dfs, '*.csv'))  
# return a list of csv in the path
# each csv name represents a candidate drug name?

dfs = []
for filename_i in dfs_filenames:
    dfs.append(pd.read_csv(filename_i))
    
dfs = pd.concat(dfs, axis=0)

In [7]:
# validation of the data set

a_0 = dfs.filename_new.tolist()
a_0_new = ['/mnt/sdf/training_data/' + i for i in a_0]
b_0 = glob.glob(os.path.join(path_to_images, '*.tif'))

# more file names than actual image files

missing_files = list(set(a_0_new) - set(b_0))
missing_files = [re.sub('/mnt/sdf/training_data/', '',i) for i in missing_files]
missing_files += ['caroline_2018 fixations_4 - april_cmap 5 - fix 15-3-18_3 - doxylamine_40x_6-4-18_40x_assay plate 0_3_2018.04.06.20.10.49_d - 4(fld 28 wv red - cy5).tif']

In [8]:
dfs = dfs[~(dfs.filename_new.isin(missing_files))].copy()

In [9]:
dfs.treatment.value_counts()

ab     8264
veh    8260
Name: treatment, dtype: int64

# Splitting images

In [10]:
import torch
from torch.utils import data
import torchvision
import cv2
from torchvision import transforms

In [13]:
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

In [23]:
transform1 = transforms.ToTensor()

In [24]:
class TileImageList(ImageList):
    def __init__(self, segments_per_image=4, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.segments_per_image = segments_per_image

        
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.sizes)*self.segments_per_image
    
    def get_image_segment(self, full_image, segment_idx):
        a = full_image.chunk(2, dim=1)
        if segment_idx < 3:
            b = a[0].chunk(2, dim=2)
            return b[segment_idx-1]
        else: 
            b = a[1].chunk(2, dim=2)
            return b[segment_idx-3]

    def get(self, i):
        #segment_idx = self.segment_idxs[i]
        segment_idx = i%self.segments_per_image
        image_ID = i//self.segments_per_image
#         fn = super().get(image_ID)   # return self.items[i]
#         self.open(fn)
        image = self.items[image_ID]
        image = pil_loader(image)  #cv2.imread('/mnt/sdf/training_data/'+image)
        image = transform1(image)

        res = self.get_image_segment(image, segment_idx)

        self.sizes[i] = res.size
        return res

In [18]:
imlist = ImageList.from_df(dfs[['filename_new','treatment']], path='/mnt/sdf/training_data')
imlist

ImageList (16524 items)
Image (3, 2048, 2048),Image (3, 2048, 2048),Image (3, 2048, 2048),Image (3, 2048, 2048),Image (3, 2048, 2048)
Path: /mnt/sdf/training_data

In [19]:
imdata = TileImageList.from_df(dfs[['filename_new','treatment']], path='/mnt/sdf/training_data')
imdata

TileImageList (16524 items)
tensor([[[0.4039, 0.4431, 0.4588,  ..., 0.3725, 0.4275, 0.4078],
         [0.4431, 0.4431, 0.4431,  ..., 0.4431, 0.3765, 0.3725],
         [0.4314, 0.4745, 0.4353,  ..., 0.3804, 0.4078, 0.4000],
         ...,
         [0.5373, 0.6549, 0.7569,  ..., 0.6118, 0.5608, 0.6157],
         [0.5412, 0.6392, 0.7647,  ..., 0.5333, 0.5294, 0.6157],
         [0.5412, 0.5961, 0.8353,  ..., 0.5608, 0.4627, 0.6314]],

        [[0.4039, 0.4431, 0.4588,  ..., 0.3725, 0.4275, 0.4078],
         [0.4431, 0.4431, 0.4431,  ..., 0.4431, 0.3765, 0.3725],
         [0.4314, 0.4745, 0.4353,  ..., 0.3804, 0.4078, 0.4000],
         ...,
         [0.5373, 0.6549, 0.7569,  ..., 0.6118, 0.5608, 0.6157],
         [0.5412, 0.6392, 0.7647,  ..., 0.5333, 0.5294, 0.6157],
         [0.5412, 0.5961, 0.8353,  ..., 0.5608, 0.4627, 0.6314]],

        [[0.4039, 0.4431, 0.4588,  ..., 0.3725, 0.4275, 0.4078],
         [0.4431, 0.4431, 0.4431,  ..., 0.4431, 0.3765, 0.3725],
         [0.4314, 0.4745, 0.43

In [20]:
split_imdata = imdata.split_by_rand_pct()

In [22]:
imdata.show_batch()

AttributeError: 'TileImageList' object has no attribute 'show_batch'